## 1) Extraction
The first step in the ETL process is to extract data from various source systems. In this case, we will extract data from UC Irvine's Machine Learning Repository. To do this, we will use the `ucimlrepo` package to fetch the dataset.

In [102]:
# Load necessary Libraries
import pandas as pd

# Load the data
data = pd.read_csv("Data/online_retail_features.csv")
data.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


### a) Description of the data

In [103]:
# Describe the data
print(f"This is a description of the data:\n{data.info()}")

# Check for missing values
print(f"Missing values in each column:\n{data.isnull().sum()}")

# Check for duplicates
print(f"Duplicate rows:\n{data.duplicated().sum()}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Description  540455 non-null  object 
 1   Quantity     541909 non-null  int64  
 2   InvoiceDate  541909 non-null  object 
 3   UnitPrice    541909 non-null  float64
 4   CustomerID   406829 non-null  float64
 5   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 24.8+ MB
This is a description of the data:
None
Missing values in each column:
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
Duplicate rows:
6007


- The data has 6 records and 541909 records.
- All columns are in the correct format apart from `InvoiceDate` which is in string format and should be converted to datetime, and `CustomerID` which is in float format and should be converted to string.
- The data contains some missing values in the `CustomerID` and `Description` columns.
- There are duplicate rows in the data.

### b) Data Cleaning
This process will involve:
- Dropping the missing values since they are not significant enough to impute.
- Converting the `InvoiceDate` column to datetime format.
- Converting the `CustomerID` column to string format.
- Removing duplicate rows.


In [104]:
# Converting InvoiceDate to datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Converting CustomerID to string
data['CustomerID'] = data['CustomerID'].astype(str)

# Removing duplicate rows
data = data.drop_duplicates()

# Remove missing values in specific columns
data = data.dropna(subset=['CustomerID', 'Description'])

# Resetting the index
data = data.reset_index(drop=True)
print(f"Data size after cleaning: {data.shape}")
data.head()

Data size after cleaning: (534532, 6)


,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## 2) Transformation
The transformation process will involve:
- Create dimensions like extract where you group by `CustomerID` to create customer summary.
- Creating new calculated columns: `TotalPrice` = `Quantity` * `UnitPrice`
- Filtering data to the sales of the year. The entire year 2011.
- Handle outliers by removing values whose `Quantity` < 0 and `UnitPrice` < 0

In [105]:
# Last value of invoice
last_invoice_date = data['InvoiceDate'].max()
print(f"Last value of invoice: {last_invoice_date}")

Last value of invoice: 2011-12-09 12:50:00


In [106]:
# Create a TotalPrice column
data['TotalPrice'] = data['Quantity'] * data['UnitPrice']

# Filter data for the last year
data = data[data['InvoiceDate'] >= '2011-01-01']

# Remove outliers
data = data[(data['Quantity'] > 0) & (data['UnitPrice'] > 0)]
print(f"Data size after transformations: {data.shape}")
data.describe()

Data size after transformations: (483353, 7)


,Quantity,InvoiceDate,UnitPrice,TotalPrice
count,483353.000000,483353,483353.000000,483353.000000
mean,10.785327,2011-07-22 04:04:34.842113280,3.842620,20.307545
min,1.000000,2011-01-04 10:00:00,0.001000,0.001000
25%,1.000000,2011-04-21 19:51:00,1.250000,3.900000
50%,4.000000,2011-08-05 16:34:00,2.080000,9.950000
75%,12.000000,2011-10-25 12:11:00,4.130000,17.700000
max,80995.000000,2011-12-09 12:50:00,11062.060000,168469.600000
std,162.491437,NaN,31.563522,281.680944


In [107]:
# Creating customer summary
customer_summary = data.groupby('CustomerID').agg(
    TotalSales=('TotalPrice', 'sum'),
    AverageSales=('TotalPrice', 'mean'),
    PurchaseCount=('InvoiceDate', 'nunique'),
    FirstPurchase=('InvoiceDate', 'min'),
    LastPurchase=('InvoiceDate', 'max'),
    Country=('Country', 'first')
).reset_index()
customer_summary.head()

,CustomerID,TotalSales,AverageSales,PurchaseCount,FirstPurchase,LastPurchase,Country
0,12346.0,77183.60,77183.600000,1,2011-01-18 10:01:00,2011-01-18 10:01:00,United Kingdom
1,12347.0,3598.21,23.829205,6,2011-01-26 14:30:00,2011-12-07 15:52:00,Iceland
2,12348.0,904.44,64.602857,3,2011-01-25 10:42:00,2011-09-25 13:13:00,Finland
3,12349.0,1757.55,24.076027,1,2011-11-21 09:51:00,2011-11-21 09:51:00,Italy
4,12350.0,334.40,19.670588,1,2011-02-02 16:01:00,2011-02-02 16:01:00,Norway


In [108]:
# Saving the transformed data to a CSV file
data.to_csv('Data/transformed_data.csv', index=False)

## 3) Loading Data into SQLite Database

In this stage, we will load the transformed data into a SQLite database. We will:
1. Create a database file (retail_dw.db)
2. Create dimension tables (CustomerDim, ProductDim, TimeDim)
3. Create a fact table (SalesFact)
4. Load the transformed data into these tables

In [110]:
# Import necessary libraries
import sqlite3
import pandas as pd
import os

# Load the transformed data
data_path = os.path.join('Data', 'transformed_data.csv')
df = pd.read_csv(data_path)

print(f"Loaded transformed data with {df.shape[0]} rows and {df.shape[1]} columns")
print("Data columns:", df.columns.tolist())
df.head()

Loaded transformed data with 483353 rows and 7 columns
Data columns: ['Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', 'TotalPrice']


,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,JUMBO BAG PINK POLKADOT,10,2011-01-04 10:00:00,1.95,13313.0,United Kingdom,19.5
1,BLUE POLKADOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom,10.5
2,RED RETROSPOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom,10.5
3,RECYCLING BAG RETROSPOT,5,2011-01-04 10:00:00,2.10,13313.0,United Kingdom,10.5
4,RED RETROSPOT SHOPPER BAG,10,2011-01-04 10:00:00,1.25,13313.0,United Kingdom,12.5


In [111]:
# Create SQLite database
db_path = 'retail_dw.db'

# Connect to SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create dimension tables
# Customer Dimension Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS CustomerDim (
    CustomerID INTEGER PRIMARY KEY,
    Country TEXT
)
''')

# Product Dimension Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS ProductDim (
    ProductID INTEGER PRIMARY KEY AUTOINCREMENT,
    Description TEXT,
    UnitPrice REAL
)
''')

# Time Dimension Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS TimeDim (
    TimeID INTEGER PRIMARY KEY AUTOINCREMENT,
    InvoiceDate DATETIME,
    Year INTEGER,
    Month INTEGER,
    Day INTEGER,
    Hour INTEGER
)
''')

# Create fact table - SalesFact
cursor.execute('''
CREATE TABLE IF NOT EXISTS SalesFact (
    SaleID INTEGER PRIMARY KEY AUTOINCREMENT,
    CustomerID INTEGER,
    ProductID INTEGER,
    TimeID INTEGER,
    Quantity INTEGER,
    TotalPrice REAL,
    FOREIGN KEY (CustomerID) REFERENCES CustomerDim(CustomerID),
    FOREIGN KEY (ProductID) REFERENCES ProductDim(ProductID),
    FOREIGN KEY (TimeID) REFERENCES TimeDim(TimeID)
)
''')

print("Database tables created successfully!")

Database tables created successfully!


In [112]:
# First, let's convert InvoiceDate to datetime format if it's not already
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Now, populate dimension tables

# 1. CustomerDim - Extract unique customers
customers = df[['CustomerID', 'Country']].drop_duplicates().dropna()
customers_list = customers.to_records(index=False).tolist()

# Insert customers into CustomerDim
cursor.executemany("INSERT OR IGNORE INTO CustomerDim (CustomerID, Country) VALUES (?, ?)", 
                  [(int(cid), country) for cid, country in customers_list])
print(f"Inserted {cursor.rowcount} unique customers into CustomerDim")

# 2. ProductDim - Extract unique products
products = df[['Description', 'UnitPrice']].drop_duplicates()
products_list = products.to_records(index=False).tolist()

# Insert products into ProductDim
cursor.executemany("INSERT INTO ProductDim (Description, UnitPrice) VALUES (?, ?)", products_list)
print(f"Inserted {cursor.rowcount} unique products into ProductDim")

# 3. TimeDim - Extract unique dates with components
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['Hour'] = df['InvoiceDate'].dt.hour

times = df[['InvoiceDate', 'Year', 'Month', 'Day', 'Hour']].drop_duplicates()
times_list = times.to_records(index=False).tolist()

# Insert times into TimeDim
cursor.executemany(
    "INSERT INTO TimeDim (InvoiceDate, Year, Month, Day, Hour) VALUES (?, ?, ?, ?, ?)",
    times_list
)
print(f"Inserted {cursor.rowcount} unique dates into TimeDim")

# Commit the changes to dimension tables
conn.commit()

Inserted 4219 unique customers into CustomerDim
Inserted 14096 unique products into ProductDim
Inserted 17069 unique dates into TimeDim


In [113]:
# Populate the SalesFact table
# We need to join with dimension tables to get the corresponding IDs

# First, create a dictionary to map product descriptions to ProductIDs
cursor.execute("SELECT ProductID, Description FROM ProductDim")
product_id_map = {desc: prod_id for prod_id, desc in cursor.fetchall()}

# Create a dictionary to map dates to TimeIDs
cursor.execute("SELECT TimeID, InvoiceDate FROM TimeDim")
time_id_map = {date: time_id for time_id, date in cursor.fetchall()}

# Initialize batch counter and batch size for inserting data
batch_size = 10000
batch_count = 0
total_rows = 0

# Process the dataframe in batches to avoid memory issues
for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i+batch_size]
    
    # Prepare data for insertion
    sales_data = []
    for _, row in batch.iterrows():
        if pd.isna(row['CustomerID']):
            continue  # Skip rows with missing CustomerID
            
        product_id = product_id_map.get(row['Description'])
        time_id = time_id_map.get(str(row['InvoiceDate']))
        
        if product_id is not None and time_id is not None:
            sales_data.append((
                int(row['CustomerID']),
                product_id,
                time_id,
                int(row['Quantity']),
                float(row['TotalPrice'])
            ))
    
    # Insert data into SalesFact
    if sales_data:
        cursor.executemany(
            "INSERT INTO SalesFact (CustomerID, ProductID, TimeID, Quantity, TotalPrice) VALUES (?, ?, ?, ?, ?)",
            sales_data
        )
        
        batch_count += 1
        total_rows += len(sales_data)
        print(f"Batch {batch_count}: Inserted {len(sales_data)} rows into SalesFact")

# Commit the changes and close the connection
conn.commit()
print(f"\nTotal: Inserted {total_rows} rows into SalesFact table")
print(f"Database created successfully at: {os.path.abspath(db_path)}")

# Close the database connection
conn.close()


Total: Inserted 0 rows into SalesFact table
Database created successfully at: c:\Users\admin\OneDrive\Desktop\School\Summer Sem 2025\Data Warehousing\END SEMESTER PROJECT\DSA-2040_Practical_Exam_-Calvin_035-\Data Warehousing\ETL_Process\retail_dw.db


In [114]:
# Let's fix the issue with the fact table population
# First, reconnect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Let's check the content of TimeDim table to see how dates are stored
cursor.execute("SELECT TimeID, InvoiceDate, Year, Month, Day FROM TimeDim LIMIT 5")
print("Sample data from TimeDim:")
for row in cursor.fetchall():
    print(row)

# Let's check if we have data in the dimension tables
cursor.execute("SELECT COUNT(*) FROM CustomerDim")
customer_count = cursor.fetchone()[0]
cursor.execute("SELECT COUNT(*) FROM ProductDim")
product_count = cursor.fetchone()[0]
cursor.execute("SELECT COUNT(*) FROM TimeDim")
time_count = cursor.fetchone()[0]

print(f"\nDimension table counts:")
print(f"CustomerDim: {customer_count} rows")
print(f"ProductDim: {product_count} rows")
print(f"TimeDim: {time_count} rows")

Sample data from TimeDim:
(1, 1294135200000000000, 2011, 1, 4)
(2, 1294136520000000000, 2011, 1, 4)
(3, 1294136580000000000, 2011, 1, 4)
(4, 1294137420000000000, 2011, 1, 4)
(5, 1294137480000000000, 2011, 1, 4)

Dimension table counts:
CustomerDim: 4219 rows
ProductDim: 14096 rows
TimeDim: 17069 rows


In [115]:
# First, let's drop the existing SalesFact table and recreate it
cursor.execute("DROP TABLE IF EXISTS SalesFact")
cursor.execute('''
CREATE TABLE IF NOT EXISTS SalesFact (
    SaleID INTEGER PRIMARY KEY AUTOINCREMENT,
    CustomerID INTEGER,
    ProductID INTEGER,
    TimeID INTEGER,
    Quantity INTEGER,
    TotalPrice REAL,
    FOREIGN KEY (CustomerID) REFERENCES CustomerDim(CustomerID),
    FOREIGN KEY (ProductID) REFERENCES ProductDim(ProductID),
    FOREIGN KEY (TimeID) REFERENCES TimeDim(TimeID)
)
''')

# Create a mapping from Description to ProductID for faster lookups
cursor.execute("SELECT ProductID, Description FROM ProductDim")
product_map = {desc: pid for pid, desc in cursor.fetchall()}

# We need to work with the TimeDim data differently
# Create a mapping for dates using Year, Month, Day, Hour
date_map = {}
cursor.execute("SELECT TimeID, Year, Month, Day, Hour FROM TimeDim")
for time_id, year, month, day, hour in cursor.fetchall():
    date_key = f"{year}-{month}-{day}-{hour}"
    date_map[date_key] = time_id

# Insert data into SalesFact in batches
batch_size = 10000
total_inserted = 0

# Process in batches
for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i+batch_size]
    
    # Prepare batch data
    sales_data = []
    for _, row in batch.iterrows():
        # Skip if missing customer ID
        if pd.isna(row['CustomerID']):
            continue
        
        # Get product ID
        product_id = product_map.get(row['Description'])
        
        # Get time ID by constructing the key
        date_key = f"{row['Year']}-{row['Month']}-{row['Day']}-{row['Hour']}"
        time_id = date_map.get(date_key)
        
        # Add to batch if all mappings exist
        if product_id and time_id:
            sales_data.append((
                int(row['CustomerID']),
                product_id,
                time_id,
                int(row['Quantity']),
                float(row['TotalPrice'])
            ))
    
    # Insert batch
    if sales_data:
        cursor.executemany(
            "INSERT INTO SalesFact (CustomerID, ProductID, TimeID, Quantity, TotalPrice) VALUES (?, ?, ?, ?, ?)",
            sales_data
        )
        inserted = len(sales_data)
        total_inserted += inserted
        # print(f"Batch: Inserted {inserted} rows, total: {total_inserted}")

# Commit and close
conn.commit()
print(f"\nSuccessfully inserted {total_inserted} rows into SalesFact table")
print(f"Database schema created and populated successfully at: {os.path.abspath(db_path)}")
conn.close()

Batch: Inserted 6589 rows, total: 6589
Batch: Inserted 4719 rows, total: 11308
Batch: Inserted 6422 rows, total: 17730
Batch: Inserted 7808 rows, total: 25538
Batch: Inserted 6950 rows, total: 32488
Batch: Inserted 7890 rows, total: 40378
Batch: Inserted 7348 rows, total: 47726
Batch: Inserted 7630 rows, total: 55356
Batch: Inserted 7995 rows, total: 63351
Batch: Inserted 7358 rows, total: 70709
Batch: Inserted 8801 rows, total: 79510
Batch: Inserted 6812 rows, total: 86322
Batch: Inserted 7579 rows, total: 93901
Batch: Inserted 7295 rows, total: 101196
Batch: Inserted 8019 rows, total: 109215
Batch: Inserted 8308 rows, total: 117523
Batch: Inserted 7663 rows, total: 125186
Batch: Inserted 7257 rows, total: 132443
Batch: Inserted 8016 rows, total: 140459
Batch: Inserted 6733 rows, total: 147192
Batch: Inserted 6115 rows, total: 153307
Batch: Inserted 7020 rows, total: 160327
Batch: Inserted 7130 rows, total: 167457
Batch: Inserted 8417 rows, total: 175874
Batch: Inserted 7396 rows, tot

In [116]:
# Reconnect to the database to verify our data
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Check row counts in each table
cursor.execute("SELECT COUNT(*) FROM CustomerDim")
customer_count = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM ProductDim")
product_count = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM TimeDim")
time_count = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM SalesFact")
sales_count = cursor.fetchone()[0]

print("Final table row counts:")
print(f"CustomerDim: {customer_count} rows")
print(f"ProductDim: {product_count} rows")
print(f"TimeDim: {time_count} rows")
print(f"SalesFact: {sales_count} rows")

# Run a sample query to join fact and dimension tables
query = """
SELECT 
    c.CustomerID, 
    c.Country, 
    p.Description, 
    t.Year, 
    t.Month, 
    t.Day,
    s.Quantity, 
    s.TotalPrice
FROM SalesFact s
JOIN CustomerDim c ON s.CustomerID = c.CustomerID
JOIN ProductDim p ON s.ProductID = p.ProductID
JOIN TimeDim t ON s.TimeID = t.TimeID
LIMIT 10
"""

cursor.execute(query)
results = cursor.fetchall()

print("\nSample data from joined tables:")
for row in results:
    print(row)

# Close the connection
conn.close()

Final table row counts:
CustomerDim: 4219 rows
ProductDim: 14096 rows
TimeDim: 17069 rows
SalesFact: 366955 rows

Sample data from joined tables:
(13313, 'United Kingdom', 'JUMBO BAG PINK POLKADOT', 2011, 1, 4, 10, 19.5)
(13313, 'United Kingdom', 'BLUE POLKADOT WRAP', 2011, 1, 4, 25, 10.5)
(13313, 'United Kingdom', 'RED RETROSPOT WRAP ', 2011, 1, 4, 25, 10.5)
(13313, 'United Kingdom', 'RECYCLING BAG RETROSPOT ', 2011, 1, 4, 5, 10.5)
(13313, 'United Kingdom', 'RED RETROSPOT SHOPPER BAG', 2011, 1, 4, 10, 12.5)
(13313, 'United Kingdom', 'JUMBO BAG RED RETROSPOT', 2011, 1, 4, 10, 19.5)
(13313, 'United Kingdom', 'RED RETROSPOT CHILDRENS UMBRELLA', 2011, 1, 4, 6, 19.5)
(13313, 'United Kingdom', 'JAM MAKING SET PRINTED', 2011, 1, 4, 12, 17.4)
(13313, 'United Kingdom', 'RECIPE BOX RETROSPOT ', 2011, 1, 4, 6, 17.700000000000003)
(13313, 'United Kingdom', 'CHILDRENS APRON APPLES DESIGN', 2011, 1, 4, 8, 15.6)


In [118]:
# Establish conn
conn = sqlite3.connect(db_path)
# Extract some data: Total sales by country
query_total_sales_by_country = """SELECT c.Country, SUM(s.TotalPrice) as TotalSales
FROM SalesFact s
JOIN CustomerDim c ON s.CustomerID = c.CustomerID
GROUP BY c.Country
ORDER BY TotalSales DESC
"""
# Load the query results into a DataFrame
df_total_sales_by_country = pd.read_sql_query(query_total_sales_by_country, conn)
df_total_sales_by_country.head()

,Country,TotalSales
0,United Kingdom,6787027.064
1,Netherlands,276661.860
2,EIRE,256448.580
3,Germany,213454.660
4,France,199318.000


## Data Loading Summary

We've successfully created a SQLite database named `retail_dw.db` with a star schema design:

**Dimension Tables:**
1. **CustomerDim** - Contains customer information with 4,219 unique customers
   - CustomerID (Primary Key)
   - Country

2. **ProductDim** - Contains product information with 14,096 unique products
   - ProductID (Primary Key)
   - Description
   - UnitPrice

3. **TimeDim** - Contains time-related information with 17,069 unique date/time combinations
   - TimeID (Primary Key)
   - InvoiceDate
   - Year
   - Month
   - Day
   - Hour

**Fact Table:**
- **SalesFact** - Contains 366,955 sales transactions
   - SaleID (Primary Key)
   - CustomerID (Foreign Key)
   - ProductID (Foreign Key)
   - TimeID (Foreign Key)
   - Quantity
   - TotalPrice

This data warehouse structure allows for efficient analysis of sales data across different dimensions such as time, products, and customers. You can run SQL queries against this database to answer business questions and generate reports.

**Example SQL queries you can run:**



2. Sales trends by month:
```sql
SELECT t.Year, t.Month, SUM(s.TotalPrice) as MonthlySales
FROM SalesFact s
JOIN TimeDim t ON s.TimeID = t.TimeID
GROUP BY t.Year, t.Month
ORDER BY t.Year, t.Month
```

3. Top selling products:
```sql
SELECT p.Description, SUM(s.Quantity) as TotalQuantity
FROM SalesFact s
JOIN ProductDim p ON s.ProductID = p.ProductID
GROUP BY p.Description
ORDER BY TotalQuantity DESC
LIMIT 10
```